In [5]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import xlsxwriter
import pandas as pd
import time
import openpyxl

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [6]:
# 모델 설계 #
tf.reset_default_graph()

# ---------- RNN Network ---------- #
batch_size = 1
rnn_size = 4
learing_rate = 0.01

with tf.variable_scope("LSTM") :
    #tf.set_random_seed(seed)
    # input place holders
    data = tf.placeholder(tf.float32, [None, 4, 1])
    y_lon = tf.placeholder(tf.float32, [None, 1])
    y_lat = tf.placeholder(tf.float32, [None, 1])

    # RNN Network
    #rnn_cell = tf.contrib.rnn.BasicRNNCell(rnn_size)
    rnn_cell = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    val, _state = tf.nn.dynamic_rnn(rnn_cell, data, dtype=tf.float32)
    pred = tf.contrib.layers.fully_connected(val[:, -1], 1, activation_fn=None)

    # cost
    cost_lon = tf.reduce_mean(tf.square(pred - y_lon))
    cost_lat = tf.reduce_mean(tf.square(pred - y_lat))

    # optimizer
    optimizer = tf.train.AdamOptimizer(learing_rate)
    train_lon = optimizer.minimize(cost_lon)
    train_lat = optimizer.minimize(cost_lat)

    # RMSE / MAE
    targets = tf.placeholder(tf.float32, [None, 1])
    predictions = tf.placeholder(tf.float32, [None, 1])
    rmse = tf.sqrt(tf.reduce_mean(tf.square(targets - predictions)))
    mae = tf.reduce_mean(tf.abs(targets - predictions))

In [7]:
# 트레이닝, 테스트 진행 함수 #

def NetworkFunc(step) :
    print("step", step+1, "...")
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)

    # Training
        for step in range(lon_epoch) :
            _, step_loss_lon = sess.run([train_lon, cost_lon], feed_dict={data:x_train, y_lon:lon_train})

            #if step%500 == 0 and step != 0 :
            #    print step,", cost_lon :", step_loss_lon
    # Test
        test_lon = sess.run(pred, feed_dict={data:x_test})
        #ttrain_lon = sess.run(pred, feed_dict={data:x_train})
        #np.savetxt("file1.csv", test_lon, delimiter=",")
        #mae_lon = sess.run(mae, feed_dict={targets:lon_test, predictions: test_lon})
        #print "MAE_lon :", mae_lon
        sess.close()

    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init)

        for step in range(lat_epoch) :
            _, step_loss_lat = sess.run([train_lat, cost_lat], feed_dict={data:x_train, y_lat:lat_train})

            #if step%500 == 0 and step != 0 :
            #    print step,", cost_lat :", step_loss_lat
    # Test
        test_lat = sess.run(pred, feed_dict={data:x_test})
        #ttrain_lat = sess.run(pred, feed_dict={data:x_train})
        #np.savetxt("file2.csv", test_pred, delimiter=",")
        #mae_lat = sess.run(mae, feed_dict={targets:lat_test, predictions: test_pred})
        #print "MAE_lat :", mae_lat
        sess.close()

    return test_lon, test_lat #, ttrain_lon, ttrain_lat

In [9]:
# 데이터 불러오기 및 변환, 모델 실행 및 결과 저장 등 #

start_time = time.time()

#caseNum = '08'
#case_list = ['01', '02', '03', '04', '05', '06', '07', '08']
case_list = ['01', '05', '06', '07']
#case_list = ['01', '02']
#case_list = ['ori', caseNum]
#corr_rate = 'mid'
#rate_list = ['mid', '2pred', '3pred']
#rate_list = ['mid']
#rate_list = ['corr']

for c in case_list :
    #data_trainSet = open('./Data/cross_4attr/relTrain_case' + c + '.dat')
    #data_testSet = open('./Data/cross_4attr/relTest_case' + c + '.dat')

    data_trainSet = open('./Data/Data2016/Train_case' + c + '.dat')
    data_testSet = open('./Data/Data2016/Test_case' + c + '.dat')
    
    """if c == 'ori' :
        data_testSet = open('./Data/new_4attr/Test_case' + caseNum + '.dat')
    else :
        data_testSet = open('./Data/Autoencoder/corr_case' + caseNum + '.dat')"""

    read_data_trainSet = np.loadtxt(data_trainSet)
    read_data_testSet = np.loadtxt(data_testSet)

    data_trainSet.close()
    data_testSet.close()

    repeatCount = 50
    epoch_list = [3000]

    test_lon = 0
    test_lat = 0

    lon_train = read_data_trainSet[:,0]
    lat_train = read_data_trainSet[:,1]
    lon_test = read_data_testSet[:,0]
    lat_test = read_data_testSet[:,1]
    y_train = read_data_trainSet[:,0:2]
    y_test = read_data_testSet[:,0:2]

    lon_train = lon_train.reshape(len(np.atleast_1d(lon_train)), 1)
    lat_train = lat_train.reshape(len(np.atleast_1d(lat_train)), 1)

    y_train = y_train.reshape(len(np.atleast_1d(y_train)), 2, 1)
    x_train = read_data_trainSet[:,2:6]
    x_train = x_train.reshape(len(np.atleast_1d(x_train)), 4, 1)

    lon_test = lon_test.reshape(len(np.atleast_1d(lon_test)), 1)
    lat_test = lat_test.reshape(len(np.atleast_1d(lat_test)), 1)

    y_test = y_test.reshape(len(np.atleast_1d(y_test)), 2, 1)
    x_test = read_data_testSet[:,2:6]
    x_test = x_test.reshape(len(np.atleast_1d(x_test)), 4, 1)

    writer = pd.ExcelWriter("./Result/LSTM_case" + c + "_" + str(repeatCount) + "_"
                            + str(time.strftime("%m%d%H%M")) + ".xlsx", engine='xlsxwriter')
    
    for epoch in epoch_list :
        lon_epoch = epoch
        lat_epoch = epoch
        
        avg_lon = np.zeros((lon_test.size, 1))
        avg_lat = np.zeros((lat_test.size, 1))
        
        for count in range(repeatCount) :
            test_lon, test_lat = NetworkFunc(count)
            avg_lon += test_lon
            avg_lat += test_lat

        avg_lon /= repeatCount
        avg_lat /= repeatCount

        df = pd.DataFrame(avg_lon, columns=['Lon'])
        df1 = pd.DataFrame(avg_lat, columns=['Lat'])
        df = pd.concat([df, df1], axis=1)
        sheet_name = c + '_' + str(epoch) + '_' + str(repeatCount)
        df.to_excel(writer, sheet_name)
    
    print(" Case", c, "Complete")

writer.save()
writer.close()
end_time = time.time()

print("Time :", end_time - start_time)

step 1 ...
step 2 ...
step 3 ...
step 4 ...
step 5 ...
step 6 ...
step 7 ...
step 8 ...
step 9 ...
step 10 ...
step 11 ...
step 12 ...
step 13 ...
step 14 ...
step 15 ...
step 16 ...
step 17 ...
step 18 ...
step 19 ...
step 20 ...
step 21 ...
step 22 ...
step 23 ...
step 24 ...
step 25 ...
step 26 ...
step 27 ...
step 28 ...
step 29 ...
step 30 ...
step 31 ...
step 32 ...
step 33 ...
step 34 ...
step 35 ...
step 36 ...
step 37 ...
step 38 ...
step 39 ...
step 40 ...
step 41 ...
step 42 ...
step 43 ...
step 44 ...
step 45 ...
step 46 ...
step 47 ...
step 48 ...
step 49 ...
step 50 ...
 Case 01 Complete
step 1 ...
step 2 ...
step 3 ...
step 4 ...
step 5 ...
step 6 ...
step 7 ...
step 8 ...
step 9 ...
step 10 ...
step 11 ...
step 12 ...
step 13 ...
step 14 ...
step 15 ...
step 16 ...
step 17 ...
step 18 ...
step 19 ...
step 20 ...
step 21 ...
step 22 ...
step 23 ...
step 24 ...
step 25 ...
step 26 ...
step 27 ...
step 28 ...
step 29 ...
step 30 ...
step 31 ...
step 32 ...
step 33 ...
step